# Gerenciador de investimentos

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
# from lib import lib as l
from lib import plot as p
from lib import data as d
from lib import finance as f

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lcolors = np.array(['#13E881', '#FF5949', '#FFB84C', '#A7BEFA', '#27FEFA'])
dcolors = np.array(['#268040', '#800B00', '#A13808', '#464CC2', '#162CF2'])

## Leitura dos dados

In [3]:
dinv, dport, dport_inv, dprices = f.read_invest()
dinvest = pd.concat([dinv, dport_inv]).reset_index(drop=True)

## Fluxos de investimentos

### Histórico de fluxos

In [4]:
dflow = dinvest.groupby('ID').apply(f.calc_flows)
dflow = dflow.reset_index(level=[0, 1])
dflow.pop('level_1')
dflow['Rend'] = dflow['Rendimento anual'].apply(d.pct2int)
dflow = dflow.sort_values(by=['Rend'], ascending=False).reset_index(drop=True)
a = dflow.pop('Rend')

### Fluxos ativos

In [5]:
dflow[dflow['Ativo']]

,ID,Valor inicial,Ganho,Meses,Rendimento,Rendimento mensal,Rendimento anual,Ganho em 1 ano,Ativo,Tipo
0,100.000,1308.000,107.600,4.033,8.23%,1.98%,26.44%,374.274,True,IVVB11
1,102.000,1390.300,25.300,2.623,1.82%,0.69%,8.58%,121.401,True,IVVB11
13,1.200,10337.670,9.090,0.393,0.09%,0.22%,2.71%,280.340,True,Nu
20,13.000,10000.000,3.510,0.393,0.04%,0.09%,1.07%,107.353,True,Nu
27,14.100,1851.770,0.000,0.033,0.0%,0.0%,0.0%,0.000,True,Nu
28,108.000,21.260,0.000,0.951,0.0%,0.0%,0.0%,0.000,True,Cash
29,101.000,875.000,-18.200,3.541,-2.08%,-0.59%,-6.86%,-58.754,True,BRAX11
30,105.000,1478.000,-62.400,0.984,-4.22%,-4.29%,-40.83%,-578.048,True,IVVB11
31,103.000,957.200,-87.100,1.705,-9.1%,-5.44%,-48.81%,-424.713,True,BBSD11
32,104.000,970.300,-113.500,1.475,-11.7%,-8.09%,-63.54%,-544.434,True,BRAX11


## Performance

In [6]:
dnu = dflow[dflow['Meses'] > 0]
dnu = dnu[dnu['Tipo'] == 'Nu']

detf = dflow[dflow['Meses'] > 0]
detf = detf[detf['Tipo'] != 'Nu']

### Renda fixa

In [7]:
p.change_df_prop(f.calc_flows_overview(dnu), 28)

Valor base,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
22202,209.36,0.22%,387.693,1.75%


### Renda Variável

In [8]:
p.change_df_prop(f.calc_flows_overview(detf), 28)

Valor base,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
8578.66,-352.1,-3.94%,-2666.32,-31.08%


### Mensal

In [9]:
fig_prof = p.make_fig()
fig_prof_total = p.make_fig()

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [10]:
dret = d.pivot_agg(f.calc_flow_gains(dinvest[dinvest['Tipo'] != 'Cash']), val='ValorDelta', idx='Data', col='Tipo', agg={'ValorDelta':np.sum})
dret['Portfolio'] = dret.sum(axis=1)
cols = dret.columns[1:]
for c in cols:
    dret = d.window(dret, val=c)

In [11]:
palette1 = {'Portfolio': lcolors[0], 'BRAX11': lcolors[2], 'IVVB11': lcolors[0], 'BBSD11': lcolors[1], 'Cash': lcolors[4]}
palette2 = {'PortfolioAvg': dcolors[0], 'BRAX11Avg': dcolors[2], 'IVVB11Avg': dcolors[0], 'BBSD11Avg': dcolors[1], 'Cash': dcolors[4]}
p.plot(dret, x='Data', fig=fig_prof_total, palette=palette1, y=dret.columns[5:6], plot_type='bar')
p.plot(dret, x='Data', fig=fig_prof_total, palette=palette2, y=dret.columns[10:11], name=['Portfolio médio'])

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …